# Cytoscape Visualization using a local Neo4j Database
This notebook demonstrates the visualization of subgraphs from the [Neo4j](https://neo4j.com/) Graph Database. It uses the [py2neo](https://py2neo.org/) library to access a Neo4j database instance.

Author: Peter W. Rose (pwrose@ucsd.edu)

In [1]:
import ipycytoscape
from py2neo import Graph, Node, Relationship
import neo4j_utils

### Create Neo4j Graph

In [2]:
neo4j_utils.start()

Directories in use:
home:         /Users/Peter/GitRepositories/neo4j-ipycytoscape/notebooks/neo4j-community-4.4.9
config:       /Users/Peter/GitRepositories/neo4j-ipycytoscape/notebooks/neo4j-community-4.4.9/conf
logs:         /Users/Peter/GitRepositories/neo4j-ipycytoscape/notebooks/neo4j-community-4.4.9/logs
plugins:      /Users/Peter/GitRepositories/neo4j-ipycytoscape/notebooks/neo4j-community-4.4.9/plugins
import:       /Users/Peter/GitRepositories/neo4j-ipycytoscape/notebooks/neo4j-community-4.4.9/import
data:         /Users/Peter/GitRepositories/neo4j-ipycytoscape/notebooks/neo4j-community-4.4.9/data
certificates: /Users/Peter/GitRepositories/neo4j-ipycytoscape/notebooks/neo4j-community-4.4.9/certificates
licenses:     /Users/Peter/GitRepositories/neo4j-ipycytoscape/notebooks/neo4j-community-4.4.9/licenses
run:          /Users/Peter/GitRepositories/neo4j-ipycytoscape/notebooks/neo4j-community-4.4.9/run
Starting Neo4j.
Started neo4j (pid:1206). It is available at http://localhost:

In [3]:
graph = Graph(password="demo")

In [4]:
graph.delete_all()

In [5]:
# This graph example is from https://github.com/nicolewhite/neo4j-jupyter
nicole = Node("Person", name="Nicole", age=24)
drew = Node("Person", name="Drew", age=20)

mtdew = Node("Drink", name="Mountain Dew", calories=9000)
cokezero = Node("Drink", name="Coke Zero", calories=0)

coke = Node("Manufacturer", name="Coca Cola", url="https://www.coca-cola.com/")
pepsi = Node("Manufacturer", name="Pepsi", url="https://www.pepsi.com/")

In [6]:
graph.create(Relationship(nicole, "LIKES", cokezero))
graph.create(Relationship(nicole, "LIKES", mtdew))
graph.create(Relationship(drew, "LIKES", mtdew))
graph.create(Relationship(pepsi, "MAKES", mtdew))
graph.create(Relationship(coke, "MAKES", cokezero))

### Run a Cypher query
This query return all nodes with their relationships as a subgraph.

In [7]:
query = """
MATCH p=()-[]->() RETURN p
"""

In [8]:
subgraph = graph.run(query).to_subgraph()

### Display subgraph with Cytoscape

In [9]:
widget = ipycytoscape.CytoscapeWidget()
widget.graph.add_graph_from_neo4j(subgraph)

### Setup styles

**Nodes**

 * use the node property "name" as the display label
 * assign colors to the three types of nodes (Person, Drink, Manufacturer)

**Edges**

 * use the relationship "name" as the edge display label
 * use a font size of 12 px
 * align (autorotate) the text with the relationship arrow
 * use the triangle arrow shape (curve-style bezier is required to draw arrows!)

In [10]:
style = [{'selector': 'node', 'style': 
             {'label': 'data(name)'}},
         {'selector': 'node[label = "Person"]', 'style': 
              {'background-color': 'blue'}},
         {'selector': 'node[label = "Drink"]', 'style': 
              {'background-color': 'orange'}},
         {'selector': 'node[label = "Manufacturer"]', 'style': 
              {'background-color': 'red'}},
         {'selector': 'edge', 'style': 
             {'label': 'data(name)', 
              'font-size': '12px',
              'text-rotation': 'autorotate',
              'curve-style': 'bezier',
              'target-arrow-shape': 'triangle'}}]

In [11]:
widget.set_style(style)

When a Neo4j subgraph is added to a Cytoscape graph, a ```tooltip``` attribute is generated that contains all Neo4j node properties.

In [12]:
widget.set_tooltip_source('tooltip')

Click on a node to show the tooltip

In [13]:
widget

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'label': 'd…

In [14]:
neo4j_utils.stop()

Stopping Neo4j........ stopped.
